In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Timedelta, Timestamp
from collections import namedtuple

from IPython.core.debugger import set_trace

In [47]:
df_estacao = pd.read_csv("DATA/dados_estacoes_5min.csv",
        parse_dates=["DATE"],
        index_col=["DATE"])

df_radar = pd.read_csv("DATA/dados_radar_semHorarioVerao.csv",
                      parse_dates=['DATE'],
                      index_col=["DATE"])

df_estacao.sort_index(inplace=True)
df_radar.sort_index(inplace=True)

In [48]:
df_est = df_estacao[df_estacao > 0].dropna(how="all",axis=0).dropna(how="all",axis=1)
indices = df_est.index
eventos = []

evento = []
last_indice = indices[0]
evento.append(last_indice)
for indice in indices[1:]:
    evento.append(indice)
    # Eventos com um gap maior que 10 min são eventos distintos
    if indice - last_indice > Timedelta("10min"):
        # eventos maiores que 10 min
        if len(evento) > 1:
            eventos.append(evento)
        evento = []
    last_indice = indice
print(len(eventos))
    

1685


In [49]:
# Evento aleátorio para tests
evento = eventos[3]
evento

[Timestamp('2016-01-01 15:50:00'),
 Timestamp('2016-01-01 16:00:00'),
 Timestamp('2016-01-01 16:10:00'),
 Timestamp('2016-01-01 16:20:00'),
 Timestamp('2016-01-01 16:30:00'),
 Timestamp('2016-01-01 16:40:00'),
 Timestamp('2016-01-01 16:50:00'),
 Timestamp('2016-01-01 19:00:00')]

In [50]:
df_est.loc[evento].dropna(axis=1, how="all")

,MB_GAM1,MB_PRO1,MB_PRO2,MB_SEG1
DATE,,,,
2016-01-01 15:50:00,NaN,NaN,NaN,15.50
2016-01-01 16:00:00,NaN,3.00,NaN,2.50
2016-01-01 16:10:00,NaN,9.75,6.75,NaN
2016-01-01 16:20:00,NaN,5.50,5.75,NaN
2016-01-01 16:30:00,NaN,1.00,4.75,NaN
2016-01-01 16:40:00,0.25,NaN,NaN,0.25
2016-01-01 16:50:00,0.50,NaN,NaN,NaN
2016-01-01 19:00:00,NaN,1.75,NaN,NaN


In [51]:
df_evento = df_est.loc[evento].dropna(axis=1, how="all")
evento_estacao = df_evento.MB_PRO1.dropna()
evento_estacao

DATE
2016-01-01 16:00:00    3.00
2016-01-01 16:10:00    9.75
2016-01-01 16:20:00    5.50
2016-01-01 16:30:00    1.00
2016-01-01 19:00:00    1.75
Name: MB_PRO1, dtype: float64

In [52]:
evento_estacao.sum()

21.0

In [53]:
evento_estacao.index

DatetimeIndex(['2016-01-01 16:00:00', '2016-01-01 16:10:00',
               '2016-01-01 16:20:00', '2016-01-01 16:30:00',
               '2016-01-01 19:00:00'],
              dtype='datetime64[ns]', name='DATE', freq=None)

In [54]:
def plot_event(radar,station,name):
    plt.figure()
    plt.title(name)
    station.plot(label="STATION")
    radar.plot(label="RADAR")
    plt.legend()
    
    file_name = "IMAGENS/eventos/" + str(name)+"jpg"
    plt.savefig(file_name)

É necessario ainda separar os eventos por estação, como é possivel ver acima os eventos começam em horarios distintos por estaçoes.


In [55]:
def test_min_pluvi(serie, indice):
    test_indice = np.arange(indice - Timedelta("6h"),indice,Timedelta("10min"))
    test_min_pluvi = serie.loc[test_indice].sum()
    
    return test_min_pluvi > 1

In [56]:
from IPython.core.debugger import set_trace
df = pd.DataFrame(columns=["start", "station","duration","Pluviometro","RADAR"])
stations = df_est.columns
for station in stations:
    df_station = df_est[station]
    df_station_nonull = df_station[df_station > 0].dropna()
    indices = df_station_nonull.index
    dict_evento = {}
    evento = []
    last_indice = indices[0]
    for indice in indices[1:]:
        evento.append(last_indice)
        #set_trace()
        # Eventos com um gap maior que 6 horas são eventos distintos
        # E com menos de 1 mm de chuva
        
        
        
        diff_ = indice - evento[0]
                 
        if  indice - last_indice > Timedelta("6h") or (diff_ >= Timedelta("6h") and test_min_pluvi(df_station,indice)):
            total_event = df_station.loc[evento].sum()
              
            dict_evento["start"] = evento[0]
            dict_evento["station"] = station
            dict_evento["duration"] = (evento[-1] - evento[0]) + Timedelta("10min")
            dict_evento["Pluviometro"] = total_event
            try:
                dict_evento["RADAR"] = df_radar[station].loc[evento].sum()
                name = station + str(evento[0])
                #plot_event(df_radar[station].loc[evento],df_station.loc[evento],name)
            except KeyError:
                dict_evento["RADAR"] = np.nan

            df = df.append(dict_evento, ignore_index=True)
            evento = []
        last_indice = indice

/home/rossuna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
/home/rossuna/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  This is separate from the ipykernel package so we can avoid doing imports until


In [57]:
df.head()

,start,station,duration,Pluviometro,RADAR
0,2016-10-07 14:40:00,MB_ANHA1,00:10:00,0.75,NaN
1,2016-10-13 08:30:00,MB_ANHA1,00:10:00,0.25,0.214636
2,2016-10-14 07:20:00,MB_ANHA1,05:40:00,12.00,5.553790
3,2016-10-20 16:00:00,MB_ANHA1,05:10:00,21.00,3.895931
4,2016-10-21 00:40:00,MB_ANHA1,00:10:00,0.25,0.000000


In [58]:
#df = df[df.Pluviometro > 2.5]

In [59]:
df.reset_index(drop=True,inplace=True)

In [60]:
df["ERROR"] = df.Pluviometro - df.RADAR

In [61]:
df.describe()

,duration,Pluviometro,RADAR,ERROR
count,7676,7676.000000,5319.000000,5319.000000
mean,0 days 02:58:54.575299,4.468667,2.730135,2.341304
std,0 days 07:38:00.922334,8.884756,7.203191,8.337878
min,0 days 00:10:00,0.062500,0.000000,-73.688426
25%,0 days 00:20:00,0.250000,0.000000,0.125000
50%,0 days 01:50:00,1.375000,0.122333,0.739536
75%,0 days 04:50:00,4.250000,1.573984,2.747830
max,18 days 06:20:00,143.000000,77.844120,142.990000


In [62]:
df[df.Pluviometro == df.Pluviometro.max()]

,start,station,duration,Pluviometro,RADAR,ERROR
6398,2017-01-18 18:20:00,MB_SEG3,01:20:00,143.0,0.01,142.99


In [63]:
df.drop(labels=df.Pluviometro.idxmax(),axis=0,inplace=True)

In [64]:
df.groupby("station").describe()["ERROR"]

,count,mean,std,min,25%,50%,75%,max
station,,,,,,,,
MB_ANHA1,103.0,3.541931,12.156568,-48.711029,0.199461,0.875000,5.711830,74.307254
MB_ANHA3,70.0,-1.317516,8.460861,-57.474742,-0.826156,0.104582,0.812500,12.435524
MB_ANHA4,113.0,-0.210460,4.098325,-24.076761,0.042348,0.125000,0.562500,12.453052
MB_ANHA5,135.0,0.722974,8.271078,-73.688426,0.114487,0.909943,2.622429,13.285372
MB_ANHA6,13.0,0.453071,0.683254,-0.406759,0.062500,0.062500,0.750000,2.139308
MB_ANHA7,148.0,-2.420864,6.497225,-31.649216,-2.757408,-0.001724,0.250000,7.874757
MB_ANHA7.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MB_ANHA8,33.0,1.375581,7.419515,-28.805520,0.080130,0.546071,1.437500,25.000000
MB_ANHA8.1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
df.to_csv("DATA/eventos6h.csv",index=False)